# Proyek Akhir Machine Learning: Sistem Rekomendasi Buku

- **Nama:** Kevin Arnandes
- **Email:** kevinarnandes21@gmail.com
- **ID Dicoding:** kevinarnandes

# Data Collection

## Import libary

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


## Gathering Data

In [2]:
# Load Dataset
url_tour = "https://raw.githubusercontent.com/Vinzzztty/playground-data-analyst/main/Dataset/pariwisata_jogja/tour.csv"
url_rating = "https://raw.githubusercontent.com/Vinzzztty/playground-data-analyst/main/Dataset/pariwisata_jogja/tour_rating.csv"
url_user = "https://raw.githubusercontent.com/Vinzzztty/playground-data-analyst/main/Dataset/pariwisata_jogja/user.csv"

tour = pd.read_csv(url_tour)
rating = pd.read_csv(url_rating)
user = pd.read_csv(url_user)

print('Jumlah data tempat wisata: ', len(tour.Place_Id.unique()))
print('Jumlah data rating tempat wisata: ', len(rating.Place_Id.unique()))
print('Jumlah data user: ', len(user.User_Id.unique()))

Jumlah data tempat wisata:  126
Jumlah data rating tempat wisata:  126
Jumlah data user:  300


# Data Understanding

## Univariate Exploratory Data Analysis

### Tour Variabel

In [3]:
tour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      126 non-null    float64
 1   Place_Name    126 non-null    object 
 2   Description   126 non-null    object 
 3   Category      126 non-null    object 
 4   City          126 non-null    object 
 5   Price         126 non-null    int64  
 6   Rating        126 non-null    float64
 7   Time_Minutes  60 non-null     float64
 8   Coordinate    126 non-null    object 
 9   Latitude      126 non-null    float64
 10  Longitude     126 non-null    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 11.0+ KB


In [ ]:
tour.shape

(126, 11)

In [4]:
print("Banyak data: ", len(tour.Place_Id.unique()))
print("Kategori tempat wisata: ", tour.Category.unique())

Banyak data:  126
Kategori tempat wisata:  ['Taman Hiburan' 'Budaya' 'Cagar Alam' 'Bahari' 'Pusat Perbelanjaan']


In [5]:
tour.Category.value_counts()

Category
Taman Hiburan         36
Bahari                34
Budaya                30
Cagar Alam            23
Pusat Perbelanjaan     3
Name: count, dtype: int64

### Rating Variabel

In [ ]:
rating.head()

User_Id  Place_Id  Place_Ratings
0        1       101              4
1        1       154              2
2        1       103              3
3        1       208              5
4        1        89              3

In [6]:
rating.Place_Ratings.value_counts()

Place_Ratings
5    622
4    601
3    589
2    571
1    487
Name: count, dtype: int64

In [7]:
rating.describe()

User_Id     Place_Id  Place_Ratings
count  2870.000000  2870.000000    2870.000000
mean    152.132056   147.662021       3.104530
std      86.489316    36.950298       1.394187
min       1.000000    85.000000       1.000000
25%      78.000000   115.000000       2.000000
50%     153.000000   149.000000       3.000000
75%     227.000000   179.000000       4.000000
max     300.000000   210.000000       5.000000

In [ ]:
rating.describe()

User_Id     Place_Id  Place_Ratings
count  2870.000000  2870.000000    2870.000000
mean    152.132056   147.662021       3.104530
std      86.489316    36.950298       1.394187
min       1.000000    85.000000       1.000000
25%      78.000000   115.000000       2.000000
50%     153.000000   149.000000       3.000000
75%     227.000000   179.000000       4.000000
max     300.000000   210.000000       5.000000

In [ ]:
rating.Place_Ratings.value_counts()

Place_Ratings
5    622
4    601
3    589
2    571
1    487
Name: count, dtype: int64

### User Variabel

In [ ]:
user.head()

User_Id                   Location   Age
0        1      Semarang, Jawa Tengah  20.0
1        2         Bekasi, Jawa Barat  21.0
2        3        Cirebon, Jawa Barat  23.0
3        4         Bekasi, Jawa Barat  21.0
4        5  Lampung, Sumatera Selatan  20.0

In [ ]:
user.describe()

User_Id         Age
count  300.000000  300.000000
mean   150.500000   28.700000
std     86.746758    6.393716
min      1.000000   18.000000
25%     75.750000   24.000000
50%    150.500000   29.000000
75%    225.250000   34.000000
max    300.000000   40.000000

Rata rata umur adalah 28 tahun

# Data Pre Processing

## Menggabungkan seluruh rating wisata

In [26]:
# Menggabungkan seluruh Place_Id pada kategori Tour
tour_all = np.concatenate((
    tour.Place_Id.unique(),
    rating.Place_Id.unique(),
))

# Mengurutkan data dan menghapus data yang sama
tour_all = np.sort(np.unique(tour_all))

print('Jumlah seluruh data wisata berdasarkan Place_Id: ', len(tour_all))

Jumlah seluruh data wisata berdasarkan Place_Id:  126


## Menggabungkan Seluruh User

In [27]:
# Menggabungkan seluruh userID
user_all = np.concatenate((
    user.User_Id.unique(),
    rating.User_Id.unique(),
))

# Menghapus data yang sama kemudian mengurutkannya
user_all = np.sort(np.unique(user_all))

print('Jumlah seluruh user: ', len(user_all))

Jumlah seluruh user:  300


## Menggabungkan Dataframe Tour dengan Dataframe Rating

In [28]:
# Definisikan dataframe rating ke dalam variabel all_tour_rate
all_tour_rate = rating

all_tour_rate

User_Id  Place_Id  Place_Ratings
0           1       101              4
1           1       154              2
2           1       103              3
3           1       208              5
4           1        89              3
...       ...       ...            ...
2865      300       107              2
2866      300       193              5
2867      300       108              5
2868      300       103              5
2869      300       163              2

[2870 rows x 3 columns]

In [29]:
# Menggabungkan all_tour_rate dengan dataframe tour berdasarkan Place_Id
all_tour = pd.merge(all_tour_rate, tour[['Place_Id', 'Place_Name', 'Category', 'Rating']], on='Place_Id', how='left')

all_tour

User_Id  Place_Id  Place_Ratings                     Place_Name  \
0           1       101              4  Kampung Wisata Sosro Menduran   
1           1       154              2          Pantai Ngrawe (Mesra)   
2           1       103              3           Tugu Pal Putih Jogja   
3           1       208              5             Taman Sungai Mudal   
4           1        89              3           De Mata Museum Jogja   
...       ...       ...            ...                            ...   
2865      300       107              2              Bangsal Pagelaran   
2866      300       193              5                     Candi Sewu   
2867      300       108              5              Embung Tambakboyo   
2868      300       103              5           Tugu Pal Putih Jogja   
2869      300       163              2            Watu Mabur Mangunan   

           Category  Rating  
0            Budaya     4.0  
1            Bahari     4.5  
2     Taman Hiburan     4.7  
3        Cagar Alam     4.6  
4            Budaya     4.4  
...             ...     ...  
2865         Budaya     4.6  
2866         Budaya     4.6  
2867  Taman Hiburan     4.4  
2868  Taman Hiburan     4.7  
2869     Cagar Alam     4.5  

[2870 rows x 6 columns]

# Data Preparation

## Mencari tahu apakah ada Missing Value

In [30]:
all_tour.isnull().sum()

User_Id          0
Place_Id         0
Place_Ratings    0
Place_Name       0
Category         0
Rating           0
dtype: int64

## Melihat kategori wisata

In [31]:
all_tour.Category.unique()

array(['Budaya', 'Bahari', 'Taman Hiburan', 'Cagar Alam',
       'Pusat Perbelanjaan'], dtype=object)

## Preparation Data

In [32]:
# Membuat variabel preparation yang berisi dataframe all_tour kemudian diurutkan berdasarkan Place_Id
preparation = all_tour

preparation.sort_values('Place_Id')

User_Id  Place_Id  Place_Ratings               Place_Name  \
810        88        85              1  Taman Pintar Yogyakarta   
1503      159        85              4  Taman Pintar Yogyakarta   
392        43        85              4  Taman Pintar Yogyakarta   
2681      282        85              1  Taman Pintar Yogyakarta   
1561      165        85              4  Taman Pintar Yogyakarta   
...       ...       ...            ...                      ...   
2774      290       210              5            Pantai Congot   
1949      206       210              1            Pantai Congot   
142        17       210              4            Pantai Congot   
405        44       210              3            Pantai Congot   
1634      172       210              3            Pantai Congot   

           Category  Rating  
810   Taman Hiburan     4.5  
1503  Taman Hiburan     4.5  
392   Taman Hiburan     4.5  
2681  Taman Hiburan     4.5  
1561  Taman Hiburan     4.5  
...             ...     ...  
2774         Bahari     4.3  
1949         Bahari     4.3  
142          Bahari     4.3  
405          Bahari     4.3  
1634         Bahari     4.3  

[2870 rows x 6 columns]

### Menghapus data Duplikat

In [33]:
# Melihat data duplikat
preparation.duplicated().sum()

23

In [34]:
# Menghapus data duplikat
preparation = preparation.drop_duplicates('Place_Id')

preparation

User_Id  Place_Id  Place_Ratings                     Place_Name  \
0          1       101              4  Kampung Wisata Sosro Menduran   
1          1       154              2          Pantai Ngrawe (Mesra)   
2          1       103              3           Tugu Pal Putih Jogja   
3          1       208              5             Taman Sungai Mudal   
4          1        89              3           De Mata Museum Jogja   
..       ...       ...            ...                            ...   
367       40        87              2     Sindu Kusuma Edupark (SKE)   
382       41       175              3           Museum Ullen Sentalu   
387       42       180              5             Pantai Depok Jogja   
413       45       146              4        Bukit Wisata Pulepayung   
834       90       140              1                  Bendung Lepen   

          Category  Rating  
0           Budaya     4.0  
1           Bahari     4.5  
2    Taman Hiburan     4.7  
3       Cagar Alam     4.6  
4           Budaya     4.4  
..             ...     ...  
367  Taman Hiburan     4.2  
382         Budaya     4.7  
387         Bahari     4.3  
413     Cagar Alam     4.5  
834  Taman Hiburan     4.6  

[126 rows x 6 columns]

### Mengkonversi data series menjadi bentuk list

In [35]:
# Mengonversi data series Place_Id menjadi dalam bentuk list
destination_id = preparation['Place_Id'].tolist()

# Mengonversi data series Place_Name menjadi dalam bentuk list
destination_name = preparation['Place_Name'].tolist()

# Mengonversi data series Category menjadi dalam bentuk list
destination_category = preparation['Category'].tolist()

print(len(destination_id))
print(len(destination_name))
print(len(destination_category))

126
126
126


In [36]:
# Membuat dictionary untuk data destination_id, destination_name, destination_category
destination = pd.DataFrame({
    'id': destination_id,
    'destination_name': destination_name,
    'category': destination_category
})
destination

id               destination_name       category
0    101  Kampung Wisata Sosro Menduran         Budaya
1    154          Pantai Ngrawe (Mesra)         Bahari
2    103           Tugu Pal Putih Jogja  Taman Hiburan
3    208             Taman Sungai Mudal     Cagar Alam
4     89           De Mata Museum Jogja         Budaya
..   ...                            ...            ...
121   87     Sindu Kusuma Edupark (SKE)  Taman Hiburan
122  175           Museum Ullen Sentalu         Budaya
123  180             Pantai Depok Jogja         Bahari
124  146        Bukit Wisata Pulepayung     Cagar Alam
125  140                  Bendung Lepen  Taman Hiburan

[126 rows x 3 columns]

# Model Development dengan Content Based Filtering

In [37]:
data = destination
data.sample(5)

id              destination_name    category
40  192         Air Terjun Sri Gethuk  Cagar Alam
61  186                 Pantai Glagah      Bahari
9   185                 Pantai Krakal      Bahari
93  124  Kauman Pakualaman Yogyakarta      Budaya
17  131                  Watu Lumbung  Cagar Alam

## TF-IDF Vectorizer

In [38]:
# Inisialisasi TFIDFVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data category
tf.fit(data['category'])

# Mapping array dari fitur inedx integer ke fitur utama
tf.get_feature_names_out()

array(['alam', 'bahari', 'budaya', 'cagar', 'hiburan', 'perbelanjaan',
       'pusat', 'taman'], dtype=object)

In [39]:
# Melakukan Fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['category'])

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

(126, 8)

In [40]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.        , 0.        , 1.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.70710678],
        ...,
        [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.70710678, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.70710678]])

### Membuat dataframe tf-idf matrix

In [41]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan jenis masakan
# Baris diisi dengan nama resto

pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=data.destination_name
).sample(8, axis=1).sample(10, axis=0)

perbelanjaan  bahari     cagar      alam   hiburan  \
destination_name                                                               
Pantai Indrayanti                  0.0     1.0  0.000000  0.000000  0.000000   
Watu Goyang                        0.0     0.0  0.000000  0.000000  0.000000   
Sumur Gumuling                     0.0     0.0  0.000000  0.000000  0.707107   
Pantai Sundak                      0.0     1.0  0.000000  0.000000  0.000000   
Taman Pelangi Yogyakarta           0.0     0.0  0.000000  0.000000  0.707107   
Pantai Drini                       0.0     1.0  0.000000  0.000000  0.000000   
Keraton Yogyakarta                 0.0     0.0  0.000000  0.000000  0.000000   
Pantai Glagah                      0.0     1.0  0.000000  0.000000  0.000000   
Kebun Teh Nglinggo                 0.0     0.0  0.707107  0.707107  0.000000   
Air Terjun Kedung Pedut            0.0     0.0  0.707107  0.707107  0.000000   

                          pusat     taman  budaya  
destination_name                                   
Pantai Indrayanti           0.0  0.000000     0.0  
Watu Goyang                 0.0  0.000000     1.0  
Sumur Gumuling              0.0  0.707107     0.0  
Pantai Sundak               0.0  0.000000     0.0  
Taman Pelangi Yogyakarta    0.0  0.707107     0.0  
Pantai Drini                0.0  0.000000     0.0  
Keraton Yogyakarta          0.0  0.000000     1.0  
Pantai Glagah               0.0  0.000000     0.0  
Kebun Teh Nglinggo          0.0  0.000000     0.0  
Air Terjun Kedung Pedut     0.0  0.000000     0.0

## Cosine Similarity

### Menghitung cosine similarity pada matrix tf-idf

In [42]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.]])

### Melihat matrix kesamaan setiap destinasi

In [45]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama resto
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['destination_name'], columns=data['destination_name'])
print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap resto
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (126, 126)


destination_name             Candi Donotirto  Desa Wisata Tembi  \
destination_name                                                  
Seribu Batu Songgo Langit                0.0                0.0   
Pantai Sedahan                           0.0                0.0   
Desa Wisata Tembi                        0.0                1.0   
Bukit Wisata Pulepayung                  0.0                0.0   
Pantai Sepanjang                         0.0                0.0   
Pantai Nguluran                          0.0                0.0   
Pantai Jungwok                           0.0                0.0   
Bukit Paralayang, Watugupit              0.0                0.0   
Blue Lagoon Jogja                        0.0                1.0   
Kampung Wisata Taman Sari                0.0                1.0   

destination_name             Hutan Pinus Pengger  Bukit Paralayang, Watugupit  \
destination_name                                                                
Seribu Batu Songgo Langit                    0.0                          1.0   
Pantai Sedahan                               0.0                          0.0   
Desa Wisata Tembi                            1.0                          0.0   
Bukit Wisata Pulepayung                      0.0                          1.0   
Pantai Sepanjang                             0.0                          0.0   
Pantai Nguluran                              0.0                          0.0   
Pantai Jungwok                               0.0                          0.0   
Bukit Paralayang, Watugupit                  0.0                          1.0   
Blue Lagoon Jogja                            1.0                          0.0   
Kampung Wisata Taman Sari                    1.0                          0.0   

destination_name             Pintoe Langit Dahromo  
destination_name                                    
Seribu Batu Songgo Langit                      1.0  
Pantai Sedahan                                 0.0  
Desa Wisata Tembi                              0.0  
Bukit Wisata Pulepayung                        1.0  
Pantai Sepanjang                               0.0  
Pantai Nguluran                                0.0  
Pantai Jungwok                                 0.0  
Bukit Paralayang, Watugupit                    1.0  
Blue Lagoon Jogja                              0.0  
Kampung Wisata Taman Sari                      0.0

# Model Evaluation

## Mendapatkan Rekomendasi

#### Mendapatkan 5 Rekomendasi Tempat Wisata

In [50]:
def top_5_destination_recommendations(nama_destinasi, similarity_data=cosine_sim_df, items=data[['destination_name', 'category']], k=5):
  """
  Rekomendasi Destinasi Wisata berdasarkan kemiripan dataframe

  Parameter:
  ---
  nama_destinasi : tipe data string (str)
                Nama Destinasi (index kemiripan dataframe)
  similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan destinasi sebagai
                      indeks dan kolom
  items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
  k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
  ---


  Pada index ini, kita mengambil k dengan nilai similarity terbesar
  pada index matrix yang diberikan (i).
  """

  index = similarity_data.loc[:,nama_destinasi].to_numpy().argpartition(
      range(-1, -k, -1))

  closest = similarity_data.columns[index[-1:-(k+2):-1]]

  closest = closest.drop(nama_destinasi, errors='ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [48]:
data[data.destination_name.eq('Pasar Beringharjo')]

id   destination_name            category
110  174  Pasar Beringharjo  Pusat Perbelanjaan

In [51]:
# Mendapatkan rekomendasi destinasi yang mirip dengan Candi Sewu
user_input = 'Pasar Beringharjo'

print("Top Recommendation Destinasi:", user_input)
print(top_5_destination_recommendations(user_input))

Top Recommendation Destinasi: Pasar Beringharjo
               destination_name            category
0  Pasar Kebon Empring Bintaran  Pusat Perbelanjaan
1   Kawasan Wisata Sosrowijayan  Pusat Perbelanjaan
2         Air Terjun Sri Gethuk          Cagar Alam
3           Goa Rancang Kencono          Cagar Alam
4      Kampung Wisata Kadipaten              Budaya


#### Mendapatkan Top N-Recommendations

In [62]:
def destination_recommendations(nama_destinasi, n, similarity_data=cosine_sim_df, items=data[['destination_name', 'category']]):
    """
    Rekomendasi Destinasi Wisata berdasarkan kemiripan dataframe

    Parameter:
    ---
    nama_destinasi : tipe data string (str)
                     Nama Destinasi (index kemiripan dataframe)
    n : tipe data integer (int)
        Jumlah rekomendasi teratas yang diinginkan
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan destinasi sebagai
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    ---

    Pada index ini, kita mengambil n dengan nilai similarity terbesar
    pada index matrix yang diberikan (i).
    """
    
    index = similarity_data.loc[:,nama_destinasi].to_numpy().argpartition(
        range(-1, -n, -1))

    closest = similarity_data.columns[index[-1:-(n+2):-1]]

    closest = closest.drop(nama_destinasi, errors='ignore')

    return pd.DataFrame(closest).merge(items)

In [63]:
# Contoh penggunaan fungsi destination_recommendations
nama_destinasi = 'Pantai Parangtritis'
n = 10
rekomendasi = destination_recommendations(nama_destinasi, n, similarity_data=cosine_sim_df)
print("Rekomendasi destinasi wisata teratas untuk", nama_destinasi, "adalah:")
print(rekomendasi)

Rekomendasi destinasi wisata teratas untuk Pantai Parangtritis adalah:
              destination_name category
0   Hutan Mangrove Kulon Progo   Bahari
1                 Pantai Samas   Bahari
2                 Pantai Kukup   Bahari
3               Pantai Kesirat   Bahari
4                 Pantai Siung   Bahari
5               Pantai Greweng   Bahari
6                Pantai Congot   Bahari
7                Pantai Sundak   Bahari
8               Pantai Sedahan   Bahari
9                 Pantai Jogan   Bahari
10            Pantai Sepanjang   Bahari
